<a href="https://colab.research.google.com/github/santiagosanchez15/Project1-comp215/blob/main/Comp_215_prject_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 comp 215

**Author:** Santiago Sanchez Covarrubias

**resources**: *Think python, Claude.ai*
  - https://allendowney.github.io/ThinkPython/. Think python URL


**Objectives**
- The creation of a SLM capable of to predict the third word

**Project description**

The project will develop a SLM capable of predicting the third word.

This project will be focus not only on developing the SLM but also on documenting the process.
Starting by adding different sections, that at the end of different sections will join all the pieces together.

After the SLM has been built with feeded data, the final SLM will be created by inhereting everything from the first one, the difference is tyhat this final version will not only take the feeded data through files but aslo through the Wikimedia REST API, the perfect source for thousand of wirtten texts.

At the end of all the documentation the full code will be available.





In [2]:
import string #used for punctuation signs
from collections import Counter #used to merge and join two dictionaries
import numpy as np

# Parsed and clean function

  - get_text:
    - will get the text from the file

  - clean_text
    - will iterate through all the text, check if there are punctuations signs, remove and create a new list of words

  - bigrams -> words
    - check the anount aof times a word repeats after an specific word, then added to dictionary
      "Hello how are you"
      1 - (Hello how)
      2- (how are)
      3- (are you)
      amount of times the second word will come after the first one
      {"Hello", {"how": 2, "this": 4}} etc.


  - trigrams -> 3 words
    - same like bigrams but instead teh combination of 3 words
      next two words plus the word checking such as
      "The castle is big and made of stone"
      1 - (the castle is)
      2- (castle is big)
      3- (is big and)
      aount of times the next two words will come after the first one
      {'The': {"castle is", 3}
      {"The': {"red carpet", 2}
      etc, next two words following the first one there fore key can be tuple

#Clean text
two functions, clean a text from a file and another one to clean the text from a string

In [3]:

def clean_text_from_file(file_name: str) -> list:
  '''from a given file returns a list of strings with the texted parsed and cleaned '''

  with open(file_name, 'r') as text: #open file given
    return [word.strip(string.punctuation).lower()  for line in text for word in line.split() if word.strip(string.punctuation)] #iterate through each word and strip to get clean word

assert clean_text_from_file('sample.txt')[:2] == ['hello', 'world']
assert clean_text_from_file('sample.txt')[-1] == 'wonderful'

In [4]:
def clean_text(string_text: str) -> list:
  '''Returns list of word cleaned '''
  return [word.strip(string.punctuation).lower() for word in string_text.split() if word.strip(string.punctuation)]

In [5]:
list1 = "Hello! my? friend is you!!!!!"
print(clean_text(list1))

['hello', 'my', 'friend', 'is', 'you']


#Trigram
get trigram from given word
so for example if input is:
hello my name is santiago
then the output would be
(hello, my, name), (my, name, is), (name, is, santiago)

In [6]:
def get_trigrams(list_word: list) -> list:
  '''Returns list of trigrams '''

  return list(zip(list_word[:-2], list_word[1:-1], list_word[2:]))

In [7]:
#testing unit

print(get_trigrams('hello my name is santiago'.split()))
assert get_trigrams('hello my name is santiago'.split()) == [('hello', 'my', 'name'), ('my', 'name', 'is'), ('name', 'is', 'santiago')]

[('hello', 'my', 'name'), ('my', 'name', 'is'), ('name', 'is', 'santiago')]


#Merge_dictionaries

function that will take two dictionaries and merge the two of them adding the elements and counting the bigrams

In [8]:
def merge_dictionaries(old_dict: dict, new_dict: dict) -> dict:
  '''Returns dictionary with updated values '''

  counter1, counter2 = 0,0

  all_keys = set(old_dict.keys() | new_dict.keys()) #we crate a set to get all the keys of both dicitionaries merging them
  result = {}

  for key in all_keys: #iterate thorugh they keys of both dictionaries
    counter1 = Counter(old_dict.get(key, {})) #use Counter function to get attributes
    counter2 = Counter(new_dict.get(key, {}))
    result[key] = dict(counter1 + counter2) #add the attributes to a new dictionary form given key

  return result



In [9]:
dict1 = {("hello", 'how'): {"are": 1, "is": 2}, ("apple", 'is'): {"a": 1, "healthy": 2}, ("apples", "are"): {"my": 1, "taste": 2}}
dict2 = {("hello", 'how'): {"are": 3}, "pear": {"yummy": 1}}
new_dict = merge_dictionaries(dict1, dict2)
print(new_dict)

{'pear': {'yummy': 1}, ('apple', 'is'): {'a': 1, 'healthy': 2}, ('apples', 'are'): {'my': 1, 'taste': 2}, ('hello', 'how'): {'are': 4, 'is': 2}}


# word_frequency
this function will take a dictionary and a given list of bigrams to update the dictionary given with the values corresponding to the frequency of the words appearance

In [10]:
def word_frequency(trigrams:list) -> dict:
  '''Returns a dictionary with updated frequency of words '''

  new_dict = {}
  for key1, key2, value in trigrams: #iterate trhough every element in the list of bigrams tuples
    if (key1, key2) not in new_dict: new_dict[(key1, key2)] = {} #create a new key if the key doenst exist
    if value not in new_dict[(key1, key2)]: new_dict[(key1, key2)][value] = 1 #give a value of 1 if the value doesnt exist
    else: new_dict[(key1, key2)][value] += 1 #update the value once the word is found


  return new_dict
#try function



In [11]:
#try function above
tuple_t = ((1,2,3), (4,3,2))
print(word_frequency(tuple_t))

{(1, 2): {3: 1}, (4, 3): {2: 1}}


#word_frequency_from_file

Lets join all the functions together into a single function
it will take a file name as a paramter and return the dictionary that will be used to feed the model

In [12]:
def word_frequency_from_file(file_name: str, old_dict: dict) -> dict:
  '''Updates dictionary of frequencies from a given file '''

  text = clean_text_from_file(file_name) #get the clean text as a list
  trigrams = get_trigrams(text) #get bigrams form the zip function
  frequency = word_frequency(trigrams) #get a new dictioanry of frequencies
  return merge_dictionaries(old_dict, frequency) #returns the updated dictionary


In [13]:
#Test function
new_dict = {}
list_files = ['text1.txt', 'text2.txt']
for file in list_files:
  new_dict = word_frequency_from_file(file, new_dict)

print(new_dict)


{('name', 'is'): {'santigo': 1}, ('hello', 'my'): {'name': 1, 'friend': 1}, ('my', 'name'): {'is': 1}, ('my', 'friend'): {'is': 1}, ('friend', 'is'): {'you': 1}}


lets test the function with more complex files

In [14]:
#test function complex files
new_dict = {}
new_file_list = ['trigram_test1.txt', 'trigram_test2.txt']
for complex_file in new_file_list:
  new_dict = word_frequency_from_file(complex_file, new_dict)

new_dict

{('the', 'blue'): {'car': 2, 'truck': 1},
 ('the', 'bed'): {'i': 1},
 ('drives', 'carefully'): {'but': 1},
 ('drives', 'fast'): {'the': 1},
 ('the', 'dog'): {'sat': 2, 'lie': 1},
 ('drives', 'wildly'): {'my': 1},
 ('sit', 'down'): {'i': 1},
 ('cat', 'sat'): {'on': 2},
 ('car', 'drives'): {'fast': 1, 'carefully': 1},
 ('likes', 'the'): {'blue': 1, 'red': 1},
 ('dog', 'lie'): {'down': 1},
 ('and', 'the'): {'dog': 1},
 ('chair', 'and'): {'the': 1},
 ('red', 'car'): {'drives': 1},
 ('sat', 'on'): {'the': 4},
 ('down', 'i'): {'saw': 1},
 ('truck', 'drives'): {'slow': 1, 'wildly': 1},
 ('dog', 'sat'): {'on': 2},
 ('my', 'friend'): {'likes': 1},
 ('the', 'floor'): {'the': 1},
 ('carefully', 'but'): {'the': 1},
 ('floor', 'the'): {'cat': 1},
 ('sister', 'likes'): {'the': 1},
 ('but', 'the'): {'red': 1},
 ('my', 'sister'): {'likes': 1},
 ('the', 'mat'): {'the': 1},
 ('saw', 'the'): {'cat': 1, 'dog': 1},
 ('fast', 'the'): {'blue': 1},
 ('blue', 'car'): {'drives': 1, 'my': 1},
 ('red', 'truck'): 

#Word_frequency_from text
Ofcourse at this point of the project we can get the word frequency from file, bu twhat if we just want to copy and paste. well that is easy

In [15]:
def word_freqeuncy_from_text(given_string: str, old_dict: dict) -> dict:
  '''Updates dictionary of frequencies from a given text '''

  new_list = clean_text(given_string)
  trigrams = get_trigrams(new_list) #get bigrams form the zip function
  frequency = word_frequency(trigrams) #get a new dictioanry of frequencies
  return merge_dictionaries(old_dict, frequency) #returns the updated dictionary

In [16]:
list1 = ['The blue car drives fast. The blue truck drives slow. The red car drives carefully, but the red truck drives wildly. My friend likes the blue car. My sister likes the red truck.', 'The cat sat on the mat. The dog sat on the floor. The cat sat on the chair, and the dog sat on the bed. I saw the cat sit down. I saw the dog lie down.']
dict1 = {}
for text in list1:
  dict1 = word_freqeuncy_from_text(text, dict1)
dict1

{('the', 'blue'): {'car': 2, 'truck': 1},
 ('the', 'bed'): {'i': 1},
 ('drives', 'carefully'): {'but': 1},
 ('drives', 'fast'): {'the': 1},
 ('the', 'dog'): {'sat': 2, 'lie': 1},
 ('drives', 'wildly'): {'my': 1},
 ('sit', 'down'): {'i': 1},
 ('cat', 'sat'): {'on': 2},
 ('car', 'drives'): {'fast': 1, 'carefully': 1},
 ('likes', 'the'): {'blue': 1, 'red': 1},
 ('dog', 'lie'): {'down': 1},
 ('and', 'the'): {'dog': 1},
 ('chair', 'and'): {'the': 1},
 ('red', 'car'): {'drives': 1},
 ('sat', 'on'): {'the': 4},
 ('truck', 'drives'): {'slow': 1, 'wildly': 1},
 ('down', 'i'): {'saw': 1},
 ('dog', 'sat'): {'on': 2},
 ('my', 'friend'): {'likes': 1},
 ('the', 'floor'): {'the': 1},
 ('carefully', 'but'): {'the': 1},
 ('floor', 'the'): {'cat': 1},
 ('sister', 'likes'): {'the': 1},
 ('but', 'the'): {'red': 1},
 ('my', 'sister'): {'likes': 1},
 ('the', 'mat'): {'the': 1},
 ('saw', 'the'): {'cat': 1, 'dog': 1},
 ('fast', 'the'): {'blue': 1},
 ('blue', 'car'): {'drives': 1, 'my': 1},
 ('red', 'truck'): 

#what next?
since we are able to get the frequency of the word and what comes next, we need to do a couple of more things to....
Next fucntions will be:
  - list of possible next word -> returns a list of key of the trigram

  - get_probability -> get weight word and divide by total weight -> probability

  - get_weight_word -> weight / total weight using numpy to assign weight return dicitonary key = word, value = probability

  - get_word -> using NumPy pseduo-random numbers, get word based on the different possibilites


#List_possible_words

returns a list of all the possible word that can be chosen independently from the weight

In [17]:
def get_all_possible_words(dict_weights: dict, bigram: tuple) -> list:
  '''Returns all possible word based on bigram '''

  return list(dict_weights[bigram].keys())

In [18]:
new_dict = {}
new_file_list = ['trigram_test1.txt', 'trigram_test2.txt']
for complex_file in new_file_list:
  new_dict = word_frequency_from_file(complex_file, new_dict)

new_dict
get_all_possible_words(new_dict, ('on', 'the') )


['mat', 'floor', 'chair', 'bed']

#get_probability function
based on the total of weight return the proability of the word occurring
for example if total weight = 25 and my word occurece 5 times that 5/25 = 0.2

In [19]:
def get_probabilty(num_appearance: int, total_weight: int) -> int:
  '''Returns probability of the word to appear '''
  return num_appearance / total_weight

In [20]:
#test function above
assert round(get_probabilty(5, 140), 5) == 0.03571
assert round(get_probabilty(0, 150), 5) == 0
assert round(get_probabilty(10, 10), 5) == 1
assert round(get_probabilty(20, 154), 5) == 0.12987

#get_total_weight
iterate trhough each value to get the toal weight, return int of weight

In [21]:
def get_total_weight(dictionary : dict, key: tuple) -> int:
  '''return total weight for given key '''

  return sum((value for value in dictionary[key].values()))

In [22]:
#Testing unit

dictionary = {('hello', "how"): {'you': 5, "are": 3}, ('i', 'am'): {'your': 5, "santiago": 3, "my": 6}}
assert get_total_weight(dictionary, ('hello', 'how')) == 8
assert get_total_weight(dictionary, ('i', 'am')) == 14

#get_weighted_words
returns list of probability of the values given, respect to the key

#

In [23]:
def get_weighted_words(dictionary: dict, key: tuple) -> list:
  '''Returns a list of tuple word, dictionary '''

  total_weight = get_total_weight(dictionary, key) #get total sum of the weight

  #generator expression to get a list of tuples that will hold the word and the total weight
  return list(((word, get_probabilty(weight, total_weight)) for word, weight in dictionary[key].items() ))

In [24]:
#testing units
dictionary = {('hello', "how"): {'you': 5, "are": 3}, ('i', 'am'): {'your': 5, "santiago": 3, "my": 6}}
assert get_weighted_words(dictionary, ('hello', 'how')) == [('you', 0.625), ('are', 0.375)]
assert get_weighted_words(dictionary, ('i', 'am')) == [('your', 0.35714285714285715), ('santiago', 0.21428571428571427), ('my', 0.42857142857142855 )]

#predict_word
form the list given, return the word by given probability

In [25]:
def predict_word()

SyntaxError: expected ':' (ipython-input-841884995.py, line 1)